In [6]:
import pandas as pd

# I. Inflation

In [9]:
series_uris = {
    "CPIH" : "/economy/inflationandpriceindices/timeseries/l55o/mm23",
    "CPI" : "/economy/inflationandpriceindices/timeseries/d7g7/mm23",
    "OOH" : "economy/inflationandpriceindices/timeseries/l5p5/mm23",
}
base_url = "https://api.beta.ons.gov.uk/v1/data?uri="

In [27]:
import requests

# Function to fetch data from ONS API
def fetch_ons_data(uri):
    response = requests.get(base_url + uri)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Fetch data for each series
data_frames = {}
for series_name, uri in series_uris.items():
    data = fetch_ons_data(uri)
    if data:
        df = pd.DataFrame(data['months'])
        df['series'] = series_name
        data_frames[series_name] = df

# Combine all dataframes into one
combined_df = pd.concat(data_frames.values(), ignore_index=True)

# Convert 'date' column to datetime format
combined_df['date'] = pd.to_datetime(combined_df['date'], format='%Y %b')

# Calculate YoY change for 'OOH' series
ooh_df = combined_df[combined_df['series'] == 'OOH'].copy()
ooh_df['value'] = ooh_df['value'].astype(float).pct_change(periods=12) * 100

# replace the original 'OOH' series data with the calculated YoY change
combined_df.loc[combined_df['series'] == 'OOH', 'value'] = ooh_df['value']

# only cols we need
combined_df = combined_df[['date', 'value', 'series']]

# min date is 10 years before the max date
min_date = combined_df['date'].max() - pd.DateOffset(years=10)
combined_df = combined_df[combined_df['date'] >= min_date]

# convert value to float
combined_df['value'] = combined_df['value'].astype(float)

# divide by 100
combined_df['value'] = combined_df['value'] / 100

# save to csv
combined_df.to_csv('fig1_data.csv', index=False)

In [28]:
combined_df.head()

,date,value,series
309,2014-10-01,0.013,CPIH
310,2014-11-01,0.011,CPIH
311,2014-12-01,0.007,CPIH
312,2015-01-01,0.005,CPIH
313,2015-02-01,0.004,CPIH


# II. Bank of England Rate 

In [40]:
boe = pd.read_csv('fig2_data.csv')

# convert date from string in yyyy-mm-dd format to datetime
boe['date'] = pd.to_datetime(boe['date'], format='%Y-%m-%d')

# min date is 10 years before the max date
min_date = boe['date'].max() - pd.DateOffset(years=10)
boe = boe[boe['date'] >= min_date]

# divide by 100
boe['rate'] = boe['rate'] / 100

# save to csv
boe.to_csv('fig2_data.csv', index=False)

# III. Pay and CPIH

For this one, I just took from the ONS site

In [49]:
pay = pd.read_csv('fig3_data.csv')

In [ ]:
# rename Period to date
pay = pay.rename(columns={
    'Period': 'date',
    'Total pay (real)': 'Real Total Pay',
    'Regular pay (real)': 'Real Regular Pay',
    }
)

# convert date from string %b %Y format to datetime
pay['date'] = pd.to_datetime(pay['date'], format='%b %Y')

# min date is 10 years before the max date
min_date = pay['date'].max() - pd.DateOffset(years=10)
pay = pay[pay['date'] >= min_date]

# melt
pay = pay.melt(id_vars=['date'], var_name='series', value_name='value')

# divide by 100
pay['value'] = pay['value'] / 100

pay

,date,series,value
0,2014-09-01,Real Total Pay,0.001
1,2014-10-01,Real Total Pay,0.006
2,2014-11-01,Real Total Pay,0.007
3,2014-12-01,Real Total Pay,0.015
4,2015-01-01,Real Total Pay,0.009
...,...,...,...
358,2024-05-01,CPIH,0.028
359,2024-06-01,CPIH,0.028
360,2024-07-01,CPIH,0.031
361,2024-08-01,CPIH,0.031


In [51]:
pay.to_csv('fig3_data.csv', index=False)

In [53]:
energy_uris = {
    "Electricity": "/economy/inflationandpriceindices/timeseries/d7dt/mm23",
    "Gas": "/economy/inflationandpriceindices/timeseries/d7du/mm23",
    "Solid fuels": "/economy/inflationandpriceindices/timeseries/d7dw/mm23",
    "Liquid fuels": "/economy/inflationandpriceindices/timeseries/d7dv/mm23",
    "Domestic fuels": "/economy/inflationandpriceindices/timeseries/d7ch/mm23",
}

In [56]:
# Fetch data for each energy series
energy_data_frames = {}
for series_name, uri in energy_uris.items():
    data = fetch_ons_data(uri)
    if data:
        df = pd.DataFrame(data['months'])
        df['series'] = series_name
        energy_data_frames[series_name] = df

# Combine all energy dataframes into one
energy_combined_df = pd.concat(energy_data_frames.values(), ignore_index=True)

# Convert 'date' column to datetime format
energy_combined_df['date'] = pd.to_datetime(energy_combined_df['date'], format='%Y %b')

# Filter data to start from 10 years before the max date
min_date_energy = energy_combined_df['date'].max() - pd.DateOffset(years=10)
energy_combined_df = energy_combined_df[energy_combined_df['date'] >= min_date_energy]

# Select only the required columns
energy_combined_df = energy_combined_df[['date', 'series', 'value']]

# Convert 'value' column to float
energy_combined_df['value'] = energy_combined_df['value'].astype(float)

energy_combined_df.to_csv('fig5_data.csv', index=False)